In [4]:
import cv2

In [6]:
# Specify the path to the image
image_path = "..\Pic\Shirine.jpeg"

# Load the image
img = cv2.imread(image_path)

# Check if the image was loaded correctly
if img is None:
    print("Error: Could not load image. Please check the file path and try again.")
else:
    # New dimensions
    new_width = 800
    new_height = 600

    # Resize the image
    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_CUBIC)

    # Display the resized image
    cv2.imshow('Resized Image', resized_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


Error: Could not load image. Please check the file path and try again.


<>:2: SyntaxWarning: invalid escape sequence '\P'
<>:2: SyntaxWarning: invalid escape sequence '\P'
C:\Users\lingj\AppData\Local\Temp\ipykernel_21708\3859630380.py:2: SyntaxWarning: invalid escape sequence '\P'
  image_path = "..\Pic\Shirine.jpeg"
